<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/SparkSQL_%EC%8B%A4%EC%8A%B5_06_%EA%B3%A0%EA%B0%9D%EC%9D%98_%EC%B4%9D_%EC%A7%80%EC%B6%9C_%EA%B3%84%EC%82%B0_%EC%A0%95%EB%A0%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [ ]:
# master : 스파크 컨텍스트가 실행될 위치.
# local로 설정 : 해당 컴퓨터(로컬)에서 실행
#  yarn, mesos 같은 클러스터 관리 플랫폼을 선택
spark = SparkSession.builder.appName("Total-Spent-By-Customer").master("local[*]").getOrCreate()

In [ ]:
directory = "C:\\Users\\mhso_lec\\study_notebook\\data"
filename  = "customer-orders.csv"

lines = spark.sparkContext.textFile(f"file:///{directory}\\{filename}")
lines.collect()[:3]

['44,8602,37.19', '35,5368,65.89', '2,3391,40.64']

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

schema = StructType([
    StructField("cust_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("amount_spent", FloatType(), True)
])

In [ ]:
customersDF = spark.read.schema(schema).csv(f"file:///{directory}\\{filename}")
customersDF.show()

+-------+-------+------------+
|cust_id|item_id|amount_spent|
+-------+-------+------------+
|     44|   8602|       37.19|
|     35|   5368|       65.89|
|      2|   3391|       40.64|
|     47|   6694|       14.98|
|     29|    680|       13.08|
|     91|   8900|       24.59|
|     70|   3959|       68.68|
|     85|   1733|       28.53|
|     53|   9900|       83.55|
|     14|   1505|        4.32|
|     51|   3378|        19.8|
|     42|   6926|       57.77|
|      2|   4424|       55.77|
|     79|   9291|       33.17|
|     50|   3901|       23.57|
|     20|   6633|        6.49|
|     15|   6148|       65.53|
|     44|   8331|       99.19|
|      5|   3505|       64.18|
|     48|   5539|       32.42|
+-------+-------+------------+
only showing top 20 rows



In [ ]:
customersDF = customersDF.select("cust_id", "amount_spent")
customersDF.show()

+-------+------------+
|cust_id|amount_spent|
+-------+------------+
|     44|       37.19|
|     35|       65.89|
|      2|       40.64|
|     47|       14.98|
|     29|       13.08|
|     91|       24.59|
|     70|       68.68|
|     85|       28.53|
|     53|       83.55|
|     14|        4.32|
|     51|        19.8|
|     42|       57.77|
|      2|       55.77|
|     79|       33.17|
|     50|       23.57|
|     20|        6.49|
|     15|       65.53|
|     44|       99.19|
|      5|       64.18|
|     48|       32.42|
+-------+------------+
only showing top 20 rows



In [ ]:
totalByCustomer = customersDF.groupBy("cust_id").agg(func.round(func.sum("amount_spent"), 2).alias("total_spent"))
totalByCustomer.show()

+-------+-----------+
|cust_id|total_spent|
+-------+-----------+
|     31|    4765.05|
|     85|    5503.43|
|     65|    5140.35|
|     53|     4945.3|
|     78|    4524.51|
|     34|     5330.8|
|     81|    5112.71|
|     28|    5000.71|
|     76|    4904.21|
|     27|    4915.89|
|     26|     5250.4|
|     44|    4756.89|
|     12|    4664.59|
|     91|    4642.26|
|     22|    5019.45|
|     93|    5265.75|
|     47|     4316.3|
|      1|     4958.6|
|     52|    5245.06|
|     13|    4367.62|
+-------+-----------+
only showing top 20 rows



In [ ]:
totalByCustomersSorted = totalByCustomer.sort("total_spent")
totalByCustomersSorted.show()

+-------+-----------+
|cust_id|total_spent|
+-------+-----------+
|     45|    3309.38|
|     79|    3790.57|
|     96|    3924.23|
|     23|    4042.65|
|     99|    4172.29|
|     75|     4178.5|
|     36|    4278.05|
|     98|    4297.26|
|     47|     4316.3|
|     77|    4327.73|
|     13|    4367.62|
|     48|    4384.33|
|     49|     4394.6|
|     94|    4475.57|
|     67|    4505.79|
|     50|    4517.27|
|     78|    4524.51|
|      5|    4561.07|
|     57|     4628.4|
|     83|     4635.8|
+-------+-----------+
only showing top 20 rows



In [ ]:
# 전체 출력
totalByCustomersSorted.show(totalByCustomersSorted.count())

+-------+-----------+
|cust_id|total_spent|
+-------+-----------+
|     45|    3309.38|
|     79|    3790.57|
|     96|    3924.23|
|     23|    4042.65|
|     99|    4172.29|
|     75|     4178.5|
|     36|    4278.05|
|     98|    4297.26|
|     47|     4316.3|
|     77|    4327.73|
|     13|    4367.62|
|     48|    4384.33|
|     49|     4394.6|
|     94|    4475.57|
|     67|    4505.79|
|     50|    4517.27|
|     78|    4524.51|
|      5|    4561.07|
|     57|     4628.4|
|     83|     4635.8|
|     91|    4642.26|
|     74|    4647.13|
|     84|    4652.94|
|      3|    4659.63|
|     12|    4664.59|
|     66|    4681.92|
|     56|    4701.02|
|     21|    4707.41|
|     80|    4727.86|
|     14|    4735.03|
|     37|     4735.2|
|      7|    4755.07|
|     44|    4756.89|
|     31|    4765.05|
|     82|    4812.49|
|      4|    4815.05|
|     10|     4819.7|
|     88|    4830.55|
|     20|    4836.86|
|     89|    4851.48|
|     95|    4876.84|
|     38|    4898.46|
|     76| 

In [ ]:
spark.stop()